In [30]:
import dare
import random
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from SISA_lib import SISA

In [31]:
data = pd.read_csv("cleaned_loan.csv")
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 10 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,"Keuangan, asuransi, perbankan",Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
1,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,Lainnya,Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
2,KARYAWAN,"TAKE OVER + TOP UP KPR, RUMAH TINGGAL","Plafon (> 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,46 - 55,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN ≥ LIMIT KREDIT,KLINIK AKUPUNTIR,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,GREEN,LAYAK,Diterima
3,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (894,632,536 - 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",RUMAH TINGGAL (RUMAH KE 2/3),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN < 100% LIMIT KREDIT,PERDAGANGAN,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",RUKO/RUKAN,READY STOK,GREEN,LAYAK,Diterima
4,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,KAS DAN NON KAS ≥ LIMIT KREDIT,COLOUR SALON,CAKUPAN AREA : LOKAL,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,YELLOW,LAYAK,Diterima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal > Tipe 70","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 20 JUTA,...,>= 15%,Properti/kendaraan < 50% plafon,"INFORMASI, TELEKOMUNIKASI DAN KOMPUTER",Public listed,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
196,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),≥ 5,"MENIKAH : ≥RP 10 JUTA, <RP 20 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,Multinasional,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,Indent,GREEN,TIDAK,Ditolak
197,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,CAKUPAN AREA : LOKAL,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
198,PENGUSAHA,"Pembelian Baru / Primary, Ruko / Rukan","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,56 - 65,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,null < 50% plafon,Lainnya,Multinasional,MOU - Sertipikat Induk,RUKO/RUKAN,Indent,GREEN,TIDAK,Ditolak


In [32]:
labelEncoder = {}
for col in data.columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    labelEncoder[col] = le
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [33]:
data = data.drop_duplicates()
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [34]:
X = data.drop(["Keputusan Akhir"], axis=1)
y = data["Keputusan Akhir"]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
X_train_base = X_train.copy()
y_train_base = y_train.copy()
X_test_base = X_test.copy()

In [37]:
#SISA train and test dataset formats
X_train_SISA = X_train.values
y_train_SISA = y_train.values
X_test_SISA = X_test.values

In [38]:
#DaRE Forest train dataset
X_train_dare = np.array(X_train.values.tolist())
y_train_dare = np.array(y_train.values.tolist())
X_test_dare = np.array(X_test.values.tolist())

In [39]:
#make model for each machine learning model

In [40]:
#Base Model using RandomForest
tree_model = RandomForestClassifier(n_estimators=50,max_depth=5, random_state=42)
tree_model.fit(X_train_base, y_train_base)

RandomForestClassifier(max_depth=5, n_estimators=50, random_state=42)

In [41]:
#predict using XGBoost model
y_pred_tree = tree_model.predict(X_test_base)
accuracy_tree = accuracy_score(y_pred_tree, y_test)
print(f"Akurasi Model Decision Tree : {accuracy_tree}")

Akurasi Model Decision Tree : 0.9444444444444444


In [42]:
#SISA Model akurasi by shard
shard = 2
accuracy = 1
exp_results = pd.DataFrame(columns=['Shard', 'Accuracy'])

In [43]:
while accuracy > 0.90:
    sisa_by_shard = SISA(shard)
    sisa_by_shard .train(X_train_SISA, y_train_SISA)
    y_pred = sisa_by_shard .predict(X_test_SISA)
    accuracy = accuracy_score(y_pred, y_test)
    exp_results = pd.concat([exp_results, pd.DataFrame([{'Shard': shard, 'Accuracy': accuracy}])], ignore_index=True)
    shard += 1

In [44]:
#show sisa accuracy by shard
data_result = pd.DataFrame(exp_results)
data_result

,Shard,Accuracy
0,2,0.944444
1,3,0.944444
2,4,0.972222
3,5,0.944444
4,6,0.888889


In [45]:
#Base on result below, SISA model will use 4 shard for expriment
sisa = SISA(4)
sisa.train(X_train_SISA, y_train_SISA)

In [46]:
y_pred_SISA = sisa.predict(X_test_SISA)
accuracy_SISA = accuracy_score(y_pred_SISA, y_test)
print(f"Akurasi Model SISA : {accuracy_SISA}")

Akurasi Model SISA : 0.9722222222222222


In [47]:
#DaRE Forest Model
dafo = dare.Forest(n_estimators=50, max_depth=5, k = 5, topd = 0, random_state=42)
dafo.fit(X_train_dare, y_train_dare)

In [48]:
#Predict using DaRE Forest Model
y_pred_dare = dafo.predict(X_test_dare)
accuracy_dare = accuracy_score(y_pred_dare, y_test)
print(f"Akurasi Model DaRE Forest : {accuracy_dare}")

Akurasi Model DaRE Forest : 0.9444444444444444


In [49]:
test_proba_dare = np.array(X_test_dare)
dafo.predict_proba(test_proba_dare)

array([[0.1836144 , 0.8163856 ],
       [0.99760714, 0.00239286],
       [0.07014977, 0.92985023],
       [0.98909091, 0.01090909],
       [0.12913914, 0.87086086],
       [0.73048061, 0.26951939],
       [0.17555133, 0.82444867],
       [0.92092835, 0.07907165],
       [0.66357301, 0.33642699],
       [0.70320901, 0.29679099],
       [0.71484837, 0.28515163],
       [0.03981291, 0.96018709],
       [0.99760714, 0.00239286],
       [0.20608049, 0.79391951],
       [0.94983889, 0.05016111],
       [0.25324227, 0.74675773],
       [0.15222164, 0.84777836],
       [0.78024046, 0.21975954],
       [0.34255332, 0.65744668],
       [0.2310763 , 0.7689237 ],
       [0.74098015, 0.25901985],
       [0.68854736, 0.31145264],
       [0.15623113, 0.84376887],
       [0.20708741, 0.79291259],
       [0.99496825, 0.00503175],
       [0.1132931 , 0.8867069 ],
       [0.16573545, 0.83426455],
       [0.4047781 , 0.5952219 ],
       [0.12354137, 0.87645863],
       [0.11179821, 0.88820179],
       [0.

In [50]:
test_proba_base = np.array(X_test_base)
tree_model.predict_proba(test_proba_base)

c:\Users\pilha\anaconda3\envs\TA\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([[0.22063189, 0.77936811],
       [0.95454545, 0.04545455],
       [0.09826515, 0.90173485],
       [0.9668    , 0.0332    ],
       [0.1283157 , 0.8716843 ],
       [0.62851111, 0.37148889],
       [0.22759226, 0.77240774],
       [0.95370485, 0.04629515],
       [0.63294944, 0.36705056],
       [0.64588227, 0.35411773],
       [0.6595969 , 0.3404031 ],
       [0.03762835, 0.96237165],
       [0.98      , 0.02      ],
       [0.1757163 , 0.8242837 ],
       [0.90728241, 0.09271759],
       [0.34273632, 0.65726368],
       [0.18418988, 0.81581012],
       [0.74302504, 0.25697496],
       [0.40816461, 0.59183539],
       [0.24942968, 0.75057032],
       [0.66694104, 0.33305896],
       [0.65492287, 0.34507713],
       [0.18261055, 0.81738945],
       [0.26202152, 0.73797848],
       [0.9409639 , 0.0590361 ],
       [0.21116522, 0.78883478],
       [0.15361767, 0.84638233],
       [0.31277489, 0.68722511],
       [0.12009766, 0.87990234],
       [0.12151024, 0.87848976],
       [0.

In [51]:
#Perbandingan label untuk masing-masing model

In [52]:
comparation_pred_model = pd.DataFrame({
    'Decision Tree': y_pred_tree,
    'SISA': y_pred_SISA,
    'DaRE Forest': y_pred_dare
})

In [53]:
comparation_pred_model

,Decision Tree,SISA,DaRE Forest
0,1,1,1
1,0,0,0
2,1,1,1
3,0,0,0
4,1,1,1
5,0,0,0
6,1,1,1
7,0,0,0
8,0,0,0
9,0,0,0


#### Melakukan proses penghapusan secara random untuk masing-masing model

In [54]:
#Create dataFrame for each model
base_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
SISA_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
dare_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy', 'Time'])
comparition_model = pd.DataFrame(columns=['Jumlah data', 'Accuracy Base', 'Accuracy SISA', 'Accuracy DaRE','Time Base', 'Time SISA', 'Time DaRE'])

In [55]:
#Create function to delete base model data
def delete_base_data(X_train_base, y_train_base, index): 
    X_data = X_train.iloc[index].tolist()
    data_to_delete = X_train_base.apply(lambda row: all(elem in row.values for elem in X_data), axis=1)
    X_train_base = X_train_base[~data_to_delete]
    y_train_base = y_train_base[~data_to_delete]
    return X_train_base, y_train_base

In [56]:
#create variabel for process
deleted_index = []
jumlah_data = len(X_train)
max_random_number = len(X_train) - 1
y_pred_comparison_flag = 0

In [57]:
while len(deleted_index) < max_random_number:
    #get new index
    new_index = False
    while new_index == False:
        index = random.randint(0, max_random_number - 1)
        if index not in deleted_index:
            deleted_index.append(index)
            break
    jumlah_data -= 1
    #Base Model with decreasing dataset
    try:
        start_time_base = time.time()
        X_train_base, y_train_base = delete_base_data(X_train_base, y_train_base,index)
        tree_model.fit(X_train_base, y_train_base)
        training_time_base = time.time() - start_time_base

        #prediction using base model with new data
        y_pred_tree = tree_model.predict(X_test_base)
        accuracy_tree = accuracy_score(y_pred_tree, y_test)

        #add to dataframe
        base_model = pd.concat([base_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_tree, "Time" : training_time_base}])], ignore_index=True)
    except:
        print("Penghapusan data mencapai maksimal")
        break

    #delete elemen by sisa
    try:
        start_time_sisa = time.time()
        sisa.delete([X_train_SISA[index]])
        training_time_sisa = time.time() - start_time_sisa

        #prediction using model with deleted data
        y_pred_SISA = sisa.predict(X_test_SISA)
        accuracy_SISA = accuracy_score(y_pred_SISA, y_test)

        #add to dataframe
        SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_SISA, "Time" : training_time_sisa}])], ignore_index=True)
    except:
        print("salah satu data pada shard, sudah habis, silahkan train ulang model dari awal")
        # SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': "-", "Time" : "-"}])], ignore_index=True)
        accuracy_SISA = "-"
        training_time_sisa = "-"
    
    #delete elemen by dareForest
    try:
        start_time_dare = time.time()
        dafo.delete(index)
        training_time_dare = time.time() - start_time_dare

        #prediction using model with deleted data
        y_pred_dare = dafo.predict(X_test_dare)
        accuracy_dare = accuracy_score(y_pred_dare, y_test)

        #add to dataframe
        dare_model = pd.concat([dare_model, pd.DataFrame([{'Jumlah data': jumlah_data,'Accuracy': accuracy_dare, "Time" : training_time_dare}])], ignore_index=True)
    except Exception as e:
        print(e)

    #comparisson model
    comparition_model = pd.concat([comparition_model, pd.DataFrame([{
        'Jumlah data' : jumlah_data,
        'Accuracy Base' : accuracy_tree,
        'Accuracy SISA' : accuracy_SISA,
        'Accuracy DaRE' : accuracy_dare,
        'Time Base' : training_time_base,
        'Time SISA' : training_time_sisa,
        'Time DaRE' : training_time_dare,
    }])], ignore_index=True)

    #comparisoon y_pred for each model
    if accuracy_tree < 0.7 and y_pred_comparison_flag == 0:
        y_pred_comparison_flag = 1
        y_pred_comparation = pd.DataFrame({
            'Data ke -': jumlah_data,
            'Decision Tree': y_pred_tree,
            'SISA': y_pred_SISA,
            'DaRE Forest': y_pred_dare,
            'y_test' : y_test
        })


salah satu data pada shard, sudah habis, silahkan train ulang model dari awal
salah satu data pada shard, sudah habis, silahkan train ulang model dari awal
salah satu data pada shard, sudah habis, silahkan train ulang model dari awal
Penghapusan data mencapai maksimal


In [58]:
comparition_model

,Jumlah data,Accuracy Base,Accuracy SISA,Accuracy DaRE,Time Base,Time SISA,Time DaRE
0,143,0.944444,0.972222,0.944444,0.044960,0.032568,0.000000
1,142,0.944444,0.972222,0.944444,0.048391,0.025244,0.000000
2,141,0.944444,0.972222,0.944444,0.056816,0.032345,0.000000
3,140,0.944444,0.972222,0.944444,0.047042,0.027822,0.002513
4,139,0.944444,0.972222,0.944444,0.061129,0.036786,0.000999
...,...,...,...,...,...,...,...
137,6,0.444444,0.444444,0.638889,0.028570,0.037225,0.000000
138,5,0.444444,0.444444,0.416667,0.041430,0.037917,0.001120
139,4,0.444444,-,0.444444,0.035722,-,0.001125
140,3,0.444444,-,0.444444,0.032288,-,0.000000


In [62]:
comparition_model.to_csv("hasil_eksperiment/exp_1/hasil_eksperiment.csv", index=False)

In [59]:
#base_model.to_csv("hasil_eksperiment/exp_1/base_exp_1.csv", index=False)

In [60]:
#SISA_model.to_csv("hasil_eksperiment/exp_1/sisa_exp_1.csv", index=False)

In [61]:
#dare_model.to_csv("hasil_eksperiment/exp_1/dare_exp_1.csv", index=False)